In [1]:
from scrapy import Selector

import requests

from scrapy import Selector

import pandas as pd

from tqdm.notebook import tqdm
tqdm.pandas()

/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Loads dataframe which contains all games
all_games = pd.read_csv('../data/raw data/all_games.csv')

In [3]:
# Remove the ', [, and ] characters from the 'link' column in 'all_games.csv' so that scrapy can access the links
all_games['link'] = all_games['link'].str.replace('[', '')
all_games['link'] = all_games['link'].str.replace(']', '')
all_games['link'] = all_games['link'].str.replace("'", '')

In [4]:
# Function to scrape the score information of a game
def get_score_info(link):
    response = requests.get(link)

    sel = Selector(text = response.text)

    containers = sel.css('div.info-score')

    game_title = Selector(text = response.text).css('div.mb h1::text').get() 

    moby_score = containers.css('.mobyscore::text').get()

    critics_score = containers.css('dl dd:nth-child(4)::text').get()

    player_score = containers.css('.stars-sm::attr(data-tooltip)').get()

    if 'Xbox' in containers.css('#platformRankings a::text').getall(): # Sometimes the Xbox score is not available, this if loop sets the value to 'N/A' if it is not available
        key = containers.css('#platformRankings a::text').getall()

        values = containers.css('#platformRankings span::text').getall()

        ranking = dict(zip(key, values))

    else:
        ranking = {'Xbox': 'N/A'}


    
    return{
            'title': game_title,
            'moby_score': moby_score,
            'critics_score': critics_score,
            'player_score': player_score,
            'xbox_score': ranking['Xbox']   
        }

In [5]:
# Applies the function to all the games in the dataframe
score_dict = all_games['link'].progress_apply(get_score_info)

  0%|          | 0/1017 [00:00<?, ?it/s]

In [6]:
# Turn score_dict into a dataframe
score = pd.DataFrame(score_dict.tolist())

In [7]:
# Save the dataframe to a csv file
score.to_csv('../data/raw data/score_info.csv', index = False)